<a href="https://colab.research.google.com/github/ElanElanElanElan/DiffusingColab/blob/main/DiffusingGradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Diffusing colab by using python! the way god intended**
---



In [ ]:
#@title **Installing** and **importing** all neccesary and semi neccesary stuff
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import shutil
import time
from PIL import Image
!pip install --upgrade gradio diffusers accelerate transformers xformers safetensors
import gradio as gr
import torch
from diffusers import DiffusionPipeline,StableDiffusionPipeline,StableDiffusionXLPipeline,DPMSolverMultistepScheduler,AutoencoderTiny

In [ ]:
#@title use drive and create the setup folders
from google.colab import drive
drive.mount('/content/drive')
!mkdir Loras
!mkdir Outputs
!mkdir Plots
lora_folder = '/content/Loras'
output_folder = '/content/Outputs'
plot_folder = "/content/Plots"

In [ ]:
#@title select which useful base model and optimization to use
Enter_model = "Sdxl" # @param ["Sdxl", "1.5"]
if Enter_model == 'Sdxl':
    model_id = "stabilityai/stable-diffusion-xl-base-1.0"
    Pipeline =  StableDiffusionXLPipeline.from_pretrained(model_id, torch_dtype=torch.float16,variant="fp16", safety_checker=None, use_safetensors=True)
    Pipeline.scheduler = DPMSolverMultistepScheduler.from_config(Pipeline.scheduler.config)
else:
    model_id = "runwayml/stable-diffusion-v1-5"
    Pipeline =  StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16,variant="fp16", safety_checker=None, use_safetensors=True)
    Pipeline.scheduler = DPMSolverMultistepScheduler.from_config(Pipeline.scheduler.config)

enter_optimisation = "optimised" #@param ["none",'optimised']
if enter_optimisation == "optimised":
    Pipeline.vae = AutoencoderTiny.from_pretrained("madebyollin/taesdxl", torch_dtype=torch.float16)
    Pipeline.enable_attention_slicing()
    Pipeline.enable_vae_slicing()
    Pipeline.enable_vae_tiling()
    Pipeline.to('cuda')


In [ ]:
#@title run the cell below to define all the functions you will use (feel free to inspect)
def Loading_Lora(lora_name_string):
    lora_list = os.listdir(lora_folder)
    matching_files = [filename for filename in lora_list if lora_name_string in filename]

    if not matching_files:
        print(f"No matching files found for '{lora_name_string}' in {lora_folder}.")
        return None

    if len(matching_files) == 1:

        selected_file = matching_files[0]
        Pipeline.load_lora_weights(os.path.join(lora_folder, selected_file))
        return Pipeline
    else:
        print("more than one lora with matching names, be more specific in the naming")

def basic_generator(input_prompt: str, number_of_pics: int, steps: int, width = 1024, height = 1024, Use_Lora: bool = False, Lora_name: str = None) -> list:
    number_of_pics = int(number_of_pics)
    steps = int(steps)
    width = int(width)
    height = int(height)

    if Use_Lora and Lora_name is None:
        raise ValueError("If 'Use_Lora' is True, 'Lora_name' must be provided.")
    elif not Use_Lora and Lora_name is not None:
        raise ValueError("If 'Use_Lora' is False, 'Lora_name' should not be provided.")
    if Use_Lora is True:
        Loading_Lora(Lora_name)
        output = [Pipeline(prompt = input_prompt, num_inference_steps= steps, width = width, height = height).images for i in range(number_of_pics)]
    else:
        output = [Pipeline(prompt = input_prompt, num_inference_steps= steps, width = width, height = height).images for i in range(number_of_pics)]
    image_paths = []
    for index, image in enumerate(output):
        file_name = f"image{index}.png"
        if file_name in os.listdir(output_folder):
            file_name = f"image{len(os.listdir(output_folder))}.png"
        image[0].save(os.path.join(output_folder, file_name), "PNG")
        image_paths.append(os.path.join(output_folder,file_name))
    Pipeline.unload_lora_weights()
    return image_paths

def plot_images(image_paths:list = None):
    if image_paths is None:
        image_paths = [
            os.path.join(output_folder, filename)
            for filename in os.listdir(output_folder)
            if os.path.isfile(os.path.join(output_folder, filename))
        ]

    num_images = len(image_paths)
    num_rows = int(num_images ** 0.5)
    num_cols = (num_images + num_rows - 1) // num_rows
    fig, axes = plt.subplots(num_rows, num_cols)
    for i, ax in enumerate(axes.ravel()):
        if i < len(image_paths):
            img = Image.open(image_paths[i])
            ax.imshow(img)
            ax.axis('off')

    i = len(os.listdir(plot_folder))
    while True:
        plt_name = os.path.join(plot_folder, f"plot{i}.jpg")
        if plt_name in os.listdir(plot_folder):
            plt_name = os.path.join(plot_folder, f"plot{i+1}.jpg")
            i += 1
        else:
            break

    plt.savefig(plt_name, bbox_inches="tight")
    plt.close()




---
overview-
---


Use the *Code Cell below* to run the pipeline using the **functions, files,imports,etc** defined in the *above code cell(s)*

# For reference-

1.  **`basic_generator("anime waifu", 2,20)`** will get you 2 individual anime waifus pics made in 20 steps each in the output folder
2.  **`plot_images(basic_generator("anime waifus",12,30))`** will you get you your 12 anime waifus made with 30 steps each on a single image in the plot directory
3. simple_generator() also accepts a singular lora now, for example: `basic_generator("close up ttg style image of ttrvn by the beach in a colorful dress", 4, 20, Use_Lora = True, Lora_name = "Raven_XL-000004") `make sure lora name is in the Lora directory
4. you can provide width and height as arguements in simple generator (uses 1024 by 1024 whether sdxl or 1.5 so better change if you using 1.5)
5. using plot_images() alone would plot all images in the output folder



In [ ]:
plot_images(basic_generator("full body anime woman on a beach, solo, anime, sunny environment, joyful expression",6,25, height= 1152, width= 896))

**Utility cells below**
---



In [ ]:
#@title run the cell to copy stuff from gdrive like loras, to the appropriate folders
user_input = "Raven_XL" #@param {type:"string"}
google_drive_folder = "/content/drive/" #@param {type:"string"}
directoryto = lora_folder # @param ["lora_folder"] {type:"raw"}




full_content = os.listdir(google_drive_folder)


for file in full_content:
    if user_input in file:
        source_path = os.path.join(google_drive_folder, file)
        shutil.copy(source_path, directoryto)


In [ ]:
#@title run the **`Gradio`** interface (not completed)
#Iface = gr.Interface(fn=user_inputs, inputs= ["text","number"], outputs= "image")
#Iface.launch(debug=True)